<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Langchain/Mistral/mistral_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompting Capabilities

chat interface [Le Chat](https://chat.mistral.ai/chat).

## Helper funcs

In [ ]:
!pip install mistralai

- Notice that it's "mistralai", and not "mistral"

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

import requests
from bs4 import BeautifulSoup
import re


In [ ]:
api_key = None
dlai_endpoint = None
client = None

In [ ]:
def load_env():
    _ = load_dotenv(find_dotenv())


def load_mistral_api_key(ret_key=False):
    load_env()
    global api_key
    global dlai_endpoint
    api_key = os.getenv("MISTRAL_API_KEY")
    dlai_endpoint = os.getenv("DLAI_MISTRAL_API_ENDPOINT")

    global client
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)

    if ret_key:
        return api_key, dlai_endpoint

In [ ]:
def mistral(user_message,
            model="mistral-small-latest",
            is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model,
            messages=messages,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model,
            messages=messages)

    return chat_response.choices[0].message.content


def get_text_embedding(txt):
    global client
    embeddings_batch_response = client.embeddings(
        model="mistral-embed",
        input=txt
    )
    return embeddings_batch_response.data[0].embedding

In [ ]:
def get_web_article_text(url, file_save_name=None):

    response = requests.get(url)
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')


    tag = soup.find("div", re.compile("^prose--styled"))
    text = tag.text

    if file_save_name:
        f = open(file_save_name, "w")
        f.write(text)
        f.close()
    return text

### Load API key and helper function

In [ ]:
load_mistral_api_key()

In [ ]:
mistral("hello, what can you do?")

'Hello! I can assist you with a variety of tasks, such as answering questions, providing information, setting reminders, managing your schedule, and much more. I can also help you with research, writing, and learning. How can I assist you today?'

## Classification

In [ ]:
prompt = """
    You are a bank customer service bot.
    Your task is to assess customer intent and categorize customer
    inquiry after <<<>>> into one of the following predefined categories:

    card arrival
    change pin
    exchange rate
    country support
    cancel transfer
    charge dispute

    If the text doesn't fit into any of the above categories,
    classify it as:
    customer service

    You will only respond with the predefined category.
    Do not provide explanations or notes.

    ###
    Here are some examples:

    Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
    Category: card arrival
    Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
    Category: exchange rate
    Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
    Category: country support
    Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue.
    Category: customer service
    ###

    <<<
    Inquiry: {inquiry}
    >>>
    Category:
"""

#### Ask Mistral to check the spelling and grammar of your prompt

In [ ]:
response = mistral(f"Please correct the spelling and grammar of \
this prompt and return a text that is the same prompt,\
with the spelling and grammar fixed: {prompt}")

In [ ]:
print(response)

You are a bank customer service bot.
Your task is to assess the customer's intent and categorize the customer inquiry after "<<<>>>" into one of the following predefined categories:

* card arrival
* change PIN
* exchange rate
* country support
* cancel transfer
* charge dispute

If the text doesn't fit into any of the above categories, classify it as:

* customer service

You will only respond with the predefined category. Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
Category: card arrival

Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any

#### Try out the model

In [ ]:
mistral(
    response.format(
        inquiry="I am inquiring about the availability of your cards in the EU"
    )
)

'country support'

## Information Extraction with JSON Mode

In [ ]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker.
"""

In [ ]:
prompt = f"""
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema:

{{
        "age": {{
            "type": "integer"
        }},
        "gender": {{
            "type": "string",
            "enum": ["male", "female", "other"]
        }},
        "diagnosis": {{
            "type": "string",
            "enum": ["migraine", "diabetes", "arthritis", "acne"]
        }},
        "weight": {{
            "type": "integer"
        }},
        "smoking": {{
            "type": "string",
            "enum": ["yes", "no"]
        }}
}}
"""

In [ ]:
response = mistral(prompt, is_json=True)
print(response)

{"age": 60, "gender": "male", "diagnosis": "diabetes", "weight": 210, "smoking": "yes"}


## Personalization

In [ ]:
email = """
Dear mortgage lender,

What's your 30-year fixed-rate APR, how is it compared to the 15-year
fixed rate?

Regards,
Anna
"""

In [ ]:
prompt = f"""

You are a mortgage lender customer service bot, and your task is to
create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure
that your response is clear, concise, and directly addresses the
customer's question. Address the customer in a friendly and
professional manner. Sign the email with "Lender Customer Support."

# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In [ ]:
response = mistral(prompt)
print(response)

Dear Anna,

Thank you for reaching out. Our current 30-year fixed-rate APR is 6.484%. In comparison, our 15-year fixed-rate APR is 5.848%. As you can see, the 15-year fixed-rate has a lower APR than the 30-year fixed-rate. This is because you'll pay off the loan in half the time, resulting in less interest paid over the life of the loan.

However, keep in mind that the monthly payment for a 15-year loan will be higher than for a 30-year loan, as you're paying off the loan in a shorter period of time.

If you have any other questions or need further clarification, please don't hesitate to ask.

Best regards,
Lender Customer Support


## Summarization

- We'll use this [article](https://www.deeplearning.ai/the-batch/mistral-enhances-ai-landscape-in-europe-with-microsoft-partnership-and-new-language-models) from The Batch

In [ ]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft.

What’s new: Mistral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year). Microsoft invested $16.3 million in the French startup, and it agreed to distribute Mistral Large on its Azure platform and let Mistral AI use Azure computing infrastructure. Mistral AI makes the new models available to try for free here and to use on its La Plateforme and via custom deployments.

Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed. Like the earlier, open source Mistral 7B and Mixtral 8x7B, they can process 32,000 tokens of input context.

Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, though falling short of GPT-4. Mistral Small, which is optimized for latency and cost, achieved 72.2 percent on MMLU.
Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.
Microsoft’s investment in Mistral AI is significant but tiny compared to its $13 billion stake in OpenAI and Google and Amazon’s investments in Anthropic, which amount to $2 billion and $4 billion respectively.
Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.
Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta. The French government has touted the company as a home-grown competitor to U.S.-based leaders like OpenAI. France’s representatives in the European Commission argued on Mistral’s behalf to loosen the European Union’s AI Act oversight on powerful AI models.

Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators. The European Commission, which already was investigating Microsoft’s agreement with OpenAI for potential breaches of antitrust law, plans to investigate the new partnership as well. Members of President Emmanuel Macron’s Renaissance party criticized the deal’s potential to give a U.S. company access to European users’ data. However, other French lawmakers support the relationship.

Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to potential customers around the world. It gives the tech giant greater access to the European market. And it gives Azure customers access to a high-performance model that’s tailored to Europe’s unique regulatory environment.

We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with a leading hyperscaler is a sign of the tremendous processing and distribution power that remains concentrated in the large, U.S.-headquartered cloud companies.
"""

* And AlphaSignal newsletter from 26.07.2024




In [ ]:
newsletter = """ AlphaSignal

Hey Tess,

Welcome to today's edition of AlphaSignal, a newsletter for developers by developers.

We identify and summarize the top 1% news, papers, models, and repos in the AI industry.

IN TODAY'S SIGNAL
Read time: 4 min 48 sec

🎖️ Top News

Mistral Releases Large 2 : an Open Source Multilingual LLM w. 80+ Coding Languages


📌 CodeRabbit

Merge Code 10x faster with AI-driven code reviews


⚡️ Trending Signals

Gemini 1.5 flash is now free for all users.


Mark Zuckerberg explains why he open sourced a $500M model.


Cohere introduces rerank 3 nimble for faster search.


Deepmind’s AlphaProof scores silver in math olympiad.


OpenAI announces SearchGPT, a new AI search feature.

📌 NVIDIA

NVIDIA releases a new way to instantly use and deploy the best models including Llama 3.1 405B, 70B, and 8B


🧠 Top Papers

4D Reconstruction from a Single Video: New method enhances 3D/2D motion tracking and view synthesis from single videos.


The Llama 3 Herd of Models: The new Llama 3.1 research paper.


KAN or MLP: A Fairer Comparison: Controlled study finds MLP generally outperforms KAN across various tasks.

If you're enjoying AlphaSignal please forward this email to a colleague.

It helps us keep this content free.

TOP NEWS
Open Source

Mistral Releases Large 2: An Open Source Multilingual LLM w. 80+ Coding Languages
⇧ 2519 Likes


What's New
Mistral AI released Mistral Large 2, their largest dense model with 123 billion parameters. This model fits on a single H100 node and supports non-commercial open-weights usage. The release follows Meta's Llama 405B.



Key Specifications and Performance Metrics

123 billion parameters on a single H100 node
128k context window, supporting dozens of languages
Achieves 84% on MMLU, 8.63 on MT Bench, and 92% on HumanEval
Available on Hugging Face for research and non-commercial use
Commercial license available for deployment
Enhanced Coding and Reasoning Capabilities
Mistral Large 2 excels in coding, trained on 80+ programming languages. It matches or surpasses models like GPT-4o, Opus-3, and Llama-3 405B in coding benchmarks.



Compared to its predecessor, Mistral Large 1, it has reduced hallucinations and improved reliability, making it more dependable for complex tasks.



Improved Instruction Following and Conversation Handling
Mistral Large 2 follows instructions better and handles long multi-turn conversations. On benchmarks like Wild Bench, Arena Hard, and MT Bench, it outperforms Llama 3.1 405B and Opus-3, and matches Sonnet-3.5 and GPT-4o. This improvement makes it suitable for applications requiring precise and sustained interactions.



Multilingual Training and Performance
Trained on a significant amount of multilingual data, Mistral Large 2 excels in languages like English, French, German, Spanish, Italian, Portuguese, Chinese, Japanese, Korean, Arabic, and Hindi.



It outperforms Llama-3.1 70B and is comparable to Llama-3.1 405B in multilingual tasks, making it versatile for global applications.



Advanced Function Calling
Mistral Large 2 features enhanced function calling and retrieval skills, performing parallel and sequential function calls effectively.



Access

You can use Mistral Large 2 today via la Plateforme under the name mistral-large-2407, and test it on le Chat. Weights for the instruct model are available and are also hosted on HuggingFace.


READ MORE

CodeRabbit: Merge Code 10x faster with AI-driven code reviews
With CodeRabbit, you get:

Codebase-aware, line-by-line reviews with 1-click fixes.

Smart real-time chat for advice, code generation, and issue creation directly from review comments.

Comprehensive pull request summaries and sequence diagrams of changes.

Most installed AI app on GitHub and GitLab marketplace, loved by 1000’s of developers.



Enjoy a 7-day trial and access for OSS projects.
TRY NOW
partner with us

TRENDING SIGNALS
Language Models

Google makes Gemini 1.5 Flash free for all users
⇧ 205 Likes

Open Source

Mark Zuckerberg explains his reasoning behind open sourcing a $500 million model
⇧ 1225 Likes

Notebooks

Cohere introduces Rerank 3 Nimble: faster reranking search & retrieval-augmented generation (RAG) Systems
⇧ 242 Likes

Fine-tuning

DeepMind's new AlphaProof achieves silver medal-level score in the International Mathematical Olympiad (IMO)
⇧ 1429 Likes

Search

OpenAI announces SearchGPT: An AI search features that give you fast and timely answers with clear and relevant sources.
⇧ 3949 Likes

NVIDIA releases a new way to instantly use and deploy the best models including Llama 3.1 405B, 70B, and 8B
Their AI Foundry lets you create custom "supermodels" tailored to your needs and train them with proprietary data as well as synthetic data generated from Llama 3.1 405B.

It can handle, data curation, synthetic data generation, fine-tuning with proprietary data, accurate response retrieval, comprehensive evaluation and deployment.

Try it now ↗️

TOP PAPERS
Video Generation

4D Reconstruction from a Single Video
⇧ 1528  Likes

Problem

Reconstructing dynamic scenes from single videos is complex due to the ill-posed nature of the task. Traditional methods are limited as they require templates, function only in nearly static scenes, or cannot track full-sequence 3D motion, which makes them unsuitable for complex, moving scenes.



Solution

This approach uses SE(3) motion bases to model motion as a combination of base movements. It integrates data-driven priors like depth maps and 2D motion tracks into a unified scene representation, enhancing consistency and accuracy.



Results

The technique sets a new standard in 3D/2D motion tracking and novel view synthesis. It lowers 3D tracking error to 0.082 EPE and raises 3D tracking accuracy (within 5cm) to 43.0% on the iPhone dataset.

Language Models

The Llama 3 Herd of Models
⇧ 1411 Likes

Problem
Modern AI requires foundation models that integrate multilinguality, coding, reasoning, and tool usage. Existing models, while advanced, do not fully integrate these capabilities with high performance across various tasks.



Solution
Llama 3, a dense Transformer model with 405B parameters and a 128K token context window, addresses this need. It was pre-trained on a 15T token multilingual corpus using 3.8 × 10^25 FLOPs, significantly outscaling previous models. Llama 3 supports extensive multilingual capabilities and integrates coding, reasoning, and tool usage more seamlessly.



Results
Llama 3 matches GPT-4's performance across numerous benchmarks. On tasks like MMLU and IFEval, Llama 3.1 405B scores 87.3% and 88.6%, respectively, showing competitiveness with or superiority to other models. Smaller versions also outperform comparable models, making Llama 3.1 a leading option across size scales.

KAN

KAN or MLP: A Fairer Comparison
⇧ 685 Likes

Problem

Comparisons between Kolmogorov-Arnold Networks (KAN) and Multi-Layer Perceptrons (MLP) are unfair due to differing parameters and FLOPs.



Solution

This study controls parameters and FLOPs to fairly compare KAN and MLP across tasks like machine learning, computer vision, NLP, audio processing, and symbolic formula representation. B-spline activation's impact is explored.



Results

MLP outperformed KAN in machine learning (86.16% vs. 85.96%), computer vision (85.88% vs. 77.88%), NLP (80.45% vs. 79.95%), and audio processing (17.74% vs. 15.49%). KAN excelled only in symbolic formula representation (1.2e-3 RMSE vs. 7.4e-3). Access the code here.
"""

In [ ]:
prompt = f"""
You are a commentator. Your task is to write a report on a newsletter.
When presented with the newsletter, come up with interesting questions to ask,
and answer each question.
Afterward, combine all the information and write a report in the markdown
format.

# Newsletter:
{newsletter}

# Instructions:
## Summarize:
In clear and concise language, summarize the key points and themes
presented in the newsletter.

## Interesting Questions:
Generate three distinct and thought-provoking questions that can be
asked about the content of the newsletter. For each question:
- After "Q: ", describe the problem
- After "A: ", provide a detailed explanation of the problem addressed
in the question.
- Enclose the ultimate answer in <>.

## Write a analysis report
Using the summary and the answers to the interesting questions,
create a comprehensive report in Markdown format.
"""

In [ ]:
response = mistral(prompt)
print(response)

Summary:
The AlphaSignal newsletter highlights the top developments in the AI industry. Mistral AI has released Large 2, an open-source multilingual LLM with 80+ coding languages and improved instruction following. CodeRabbit is an AI-driven tool for faster code merging. Gemini 1.5 flash is now free for all users, and Mark Zuckerberg has open-sourced a $500M model. Cohere has introduced Rerank 3 Nimble for faster search. DeepMind's AlphaProof scored silver in the math olympiad. NVIDIA has released a new method for deploying models, including Llama 3.1. Top papers include a new method for 4D reconstruction from a single video and the Llama 3 Herd of Models. A study found MLP generally outperforms KAN across various tasks.

Interesting Questions:
1. Q: How does Mistral Large 2 compare to its predecessor and other models in terms of performance and capabilities?
   A: Mistral Large 2 has reduced hallucinations and improved reliability compared to Mistral Large 1. It also matches or surpas

## The Mistral Python client
- Below is the helper function that you imported from helper.py and used earlier in this notebook.
- For more details, check out the [Mistral AI API documentation](https://docs.mistral.ai/api/)
- To get your own Mistral AI API key to use on your own, outside of this classroom, you can create an account and go to the [console](https://console.mistral.ai/) to subscribe and create an API key.

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def mistral(user_message,
            model="mistral-small-latest",
            is_json=False):
    client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model,
            messages=messages,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model,
            messages=messages)

    return chat_response.choices[0].message.content